---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
for graph in P1_Graphs:
    degrees = sorted(nx.degree(graph).values(), reverse = True)[:10]
    path = nx.average_shortest_path_length(graph)
    cluster = nx.average_clustering(graph)
    print(degrees, path, cluster)

[100, 49, 39, 39, 38, 30, 30, 29, 27, 27] 4.099161161161161 0.03167539146454044
[13, 12, 12, 12, 12, 12, 12, 12, 12, 12] 5.089871871871872 0.5642419635919628
[6, 6, 6, 6, 6, 6, 6, 6, 6, 5] 9.378702269692925 0.4018222222222227
[92, 62, 55, 53, 51, 47, 47, 46, 46, 46] 3.1048046283934134 0.03780379975223251
[11, 9, 9, 8, 8, 8, 8, 8, 8, 8] 5.0785509568313305 0.0033037037037037037


In [6]:
def graph_identification():
    
    # Your Code Here
    # 'PA' graphs have nodes with very high degree, so there are graphs 1 and 4, 'SW_L' graphs have rather high 
    # average clustering, so there are graphs 2 and 3, 'SW_H' graphs have very low average_clustering, therefore 5.
    return(['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']) # Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [3]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [49]:
def salary_predictions():
    
    # Your Code Here
    for node in G.nodes():
        G.add_node(node, neighbors = len(G.neighbors(node)))
        G.add_node(node, promoted = sum([1 if G.node[n]['ManagementSalary'] == 1 else 0 for n in G.neighbors(node)]))
        G.add_node(node, percentage = G.node[node]['promoted'] / G.node[node]['neighbors'])
    df = pd.DataFrame(index = G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['neighbors'] = pd.Series(nx.get_node_attributes(G, 'neighbors'))
    df['promoted'] = pd.Series(nx.get_node_attributes(G, 'promoted'))
    df['percentage'] = pd.Series(nx.get_node_attributes(G, 'percentage'))
    df['salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    df_dummies = pd.get_dummies(df['department'])
    df_final = pd.concat([df.drop('department', axis = 1), df_dummies], axis = 1)
    
    df_train = df_final.dropna()
    df_test = df_final[np.isnan(df_final['salary'])]
    
    y_train = df_train['salary'].values
    df_train.drop('salary', axis = 1, inplace = True)
    df_test.drop('salary', axis = 1, inplace = True)
    
    X_train = df_train.values
    X_test = df_test.values
    
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 147)
    grid = {'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2], 'n_estimators': [50, 100, 150, 200, 250], 'max_depth': [2, 3, 4, 5]}
    gb_gd = GridSearchCV(GradientBoostingClassifier(random_state = 147), grid, cv = skf, scoring = 'roc_auc')
    gb_gd.fit(X_train, y_train)
    df_test['predicted'] = gb_gd.predict_proba(X_test)[:,1]
    return(df_test['predicted']) # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [5]:
df = future_connections
df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
df['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
df['Common Neighbors'] = df.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))
df.head()

,Future Connection,preferential attachment,jaccard,Common Neighbors
"(6, 840)",0.0,2070,0.073770,9
"(4, 197)",0.0,3552,0.015504,2
"(620, 979)",0.0,28,0.000000,0
"(519, 872)",0.0,299,0.060606,2
"(382, 423)",0.0,205,0.000000,0


In [6]:
def new_connections_predictions():
    
    # Your Code Here
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    
    df_train = future_connections.dropna()
    df_test = future_connections[np.isnan(future_connections['Future Connection'])]
    
    X_train = df_train.iloc[:,1:].values
    X_test = df_test.iloc[:,1:].values
    y_train = df_train.iloc[:,0].values
    
    scaler = StandardScaler()
    log_reg = LogisticRegression()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    log_reg.fit(X_train_scaled, y_train)
    df_test['predicted'] = log_reg.predict_proba(X_test_scaled)[:,1]
    return(df_test['predicted']) # Your Answer Here